## First an index:

In [3]:
import os
from glob import glob
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import support as sp
import pims as pm
from tqdm.auto import tqdm
from IPython.display import HTML
import pint
idx = pd.IndexSlice

tqdm.pandas()

%matplotlib inline

%reload_ext autoreload
%autoreload 2

In [4]:
# From https://stackoverflow.com/questions/2537868/sans-serif-math-with-latex-in-matplotlib
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
mpl.rcParams['text.latex.preamble'] = [
       #r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       #r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
] 

mpl.rcParams.update({'figure.dpi': 200})

mpl.rcParams.update({'font.size': 10})

ureg = pint.UnitRegistry()
column = 8.68*ureg.cm
two_columns = 18*ureg.cm

In [5]:
directory = "C:/Users/aortiza/Desktop/Tracking_20190617/"

In [6]:
index = pd.read_csv(os.path.join(directory, "index.dat"), sep = "\t", index_col = [0])
index.head()

,sigma,N particles,omega,radius,File Name
0,0.98,1.0,0.72,17.43,Test30_20190617
1,0.98,3.0,0.72,17.43,Test29_20190617
2,0.98,6.0,0.72,17.43,Test28_20190617
3,0.98,9.0,0.72,17.43,Test27_20190617
4,0.98,11.0,0.72,17.43,Test26_20190617


In [7]:
def find_file(sample, directory):
    for filename in glob(os.path.join(directory+'/*')):
        if sample["File Name"][:8] in os.path.split(filename)[1]:
            if ".dat" in filename:
                return filename

In [8]:
index["timestamp_file"] = [
    os.path.split(find_file(sample,os.path.join(directory,"Timestamps")))[-1] 
    for i,sample in index.iterrows()]

In [9]:
def read_trj(experiment):
    return pd.read_csv(os.path.join(directory,"Tracking_"+experiment["File Name"]+".dat"),
                  sep = "\t",usecols = ["x","y","frame","particle"], index_col = ["frame","particle"])
    

In [10]:
def assign_timestamps(trj, experiment):
    
    timestamp = pd.read_csv(os.path.join(directory,os.path.join("Timestamps",experiment["timestamp_file"])),
                  sep = "\t", names = ["time"])
    
    trj["time"] = timestamp.loc[trj.index.get_level_values("frame").values].values
    
    return trj

In [11]:
def recenter(trj):
    center = sp.get_center(trj)
    trj.x = trj.x-center[0]
    trj.y = trj.y-center[1]
    
    return trj
def rescale(trj,scale):
    trj.x = trj.x*scale
    trj.y = trj.y*scale
    return trj

In [12]:
def polar_coord(trj):
    trj["theta"] = np.arctan2(trj.y,trj.x)
    trj["r"] = np.sqrt(trj.x**2+trj.y**2)
    return trj

def unwrap_angle(series):
    out = series.copy(deep=True)
    out[:] = np.unwrap(series)
    return out

def unwrap_theta(trj):
    trj["theta"] = trj.theta.groupby("particle").apply(unwrap_angle)
    return trj

In [13]:
def calculate_velocities(trj):
    vel = trj.groupby("particle").diff()
    trj["omega"] = vel.theta/vel.time
    trj["vel_theta"] = trj.r*vel.theta/vel.time
    return trj

In [14]:
experiment = index.loc[3]

In [15]:
experiment

sigma                                       0.98
N particles                                    9
omega                                       0.72
radius                                     17.43
File Name                        Test27_20190617
timestamp_file    Test27_2019_06_17_16_43_11.dat
Name: 3, dtype: object

In [16]:
def get_exp_velocity(experiment):
    trj = read_trj(experiment)
    trj = assign_timestamps(trj,experiment)
    trj = recenter(trj)
    trj = rescale(trj,0.1805)
    trj = polar_coord(trj)
    trj = unwrap_theta(trj)
    trj = calculate_velocities(trj)
    return trj.filter(["r","omega","vel_theta"]).mean()

In [ ]:
omegas = index.loc[:].progress_apply(get_exp_velocity, axis=1)

In [ ]:
omegas.head()

In [ ]:
index["vel_theta"] = omegas.vel_theta
index["omega_part"] = omegas.omega

In [ ]:
index["vel_drive"] = index.omega*index.radius
index["density"] = index["N particles"] * 4 / (2*np.pi*index.radius)

In [ ]:
index["vel_theta_omega"] = -(index.omega_part-index.omega)*index.radius
index["vel_theta"] = -(index.vel_theta-index.vel_drive)

index["current"] = index.vel_theta*index.density

In [ ]:
single_particle_current = index.groupby("N particles").mean().current[1]
single_particle_current

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(column).to(ureg.inch).magnitude*np.array([1,.75]))

for sgm, curr in index.groupby(["sigma"]):
    curr_dens = curr.groupby("density").mean()
    plt.plot(np.insert(curr_dens.index.values,0,0),
             np.insert(curr_dens.current.values,0,0),"-s", label=sgm)
    
plt.legend(title = r"$\frac{\sigma}{\lambda}$",bbox_to_anchor = (1,1));